In [89]:
import requests
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Helper Functions

These functions train a model on your data and use cross validation to determine the average negative mean squared error. You do not have to edit these! However, it might make sense to add new helper functions here.

In [90]:
def scoreLinear(X, y):
    """Trains a linear regression model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    regr = linear_model.LinearRegression()
    scores = cross_val_score(regr, X, y, cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [91]:
def scoreBoosting(X, y):
    """Trains a gradient boosting model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0, loss='squared_error')
    # clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0, loss='ls') # older versions

    scores = cross_val_score(clf, X, np.ravel(y), cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [92]:
def dataframeToXy(df, predict_column, feature_columns):
    """Convert the dataframe to a format usable for the ML algorithms"""
    X = df[feature_columns].values.reshape(-1, df[feature_columns].shape[1]) # all features
    y = df[[predict_column]].values.reshape(-1, 1) # values to predict
    return X, y

In [93]:
def runScoring(df, predict_column, feature_columns):
    """This runs both algorithms to determine the scores for all given features in the dataset.
    
    Parameters: 
    df (dataframe): dataframe with Score and features
    predict_column: name of the value to predict, e.g. 'Score'
    feature_columns: list of all column names that are to be used as features
  
    Returns: 
    linear: negative mean squared error of linear regression
    boost: negative mean squared error of boosting algorithm
    """
    X, y = dataframeToXy(df, predict_column, feature_columns)
    boost = scoreBoosting(X, y)
    linear = scoreLinear(X, y)
    return linear, boost

In [94]:
def runScoringSimple(df, predict_column):
    """Alternative to function above, this takes all the columns in the 
    dataframe as features except the single column which is used for the score.
    """
    all_feature_columns = list(df.columns)
    all_feature_columns.remove(predict_column)
    return runScoring(df, predict_column, all_feature_columns)

In [95]:
def createBaselineRandom(df, predict_column):
    """creates random features and returns the average score after 20 rounds"""
    np.random.seed(0)
    baseline = df[['Score']].copy()
    averageLinearScore = 0
    averageBoostingScore = 0
    rounds = 20
    for i in range(rounds):
        baseline['feature1'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature2'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature3'] = np.random.randint(0, 100, df.shape[0])
        lin, boost = runScoring(baseline, predict_column, ['feature1', 'feature2', 'feature3'])
        averageLinearScore += lin / rounds
        averageBoostingScore += boost / rounds
    print(f'linear   : {averageLinearScore}')
    print(f'boosting : {averageBoostingScore}')

# Load Data

Recommended: Make sure you have a column named 'Score', preferably as the first column, while the name of the municipality is the index. It is fine to use your previous notebook from assignment 1 to export the data to a .csv or excel and simply import the file here, you do not have to show the code for that. Your initial dataframe should look somewhat like the one below, with more features of course.

**Data preformatting:** Since the Model only works with features that are numbers two tactics were chosen deleting values and encoding them. Since the Bezirk is already represented by number, the Kanton is always SO and PdA/Sol._2019 is always empty no new information can be gained from them. Therefore, those columns have been removed. Datum der Aufnahme on the other hand was encoded to amount of days since the first Gemeinde joined the canton. to prevent loss of a feature. Additionally missing data is replaced with the number 0.

In [96]:
df = pd.read_pickle("data/merged_data.pkl")


# Encode Datum der Aufnahme to days since first Aufnahme
df['Datum der Aufnahme'] = pd.to_datetime(df['Datum der Aufnahme'], format='%Y-%m-%d', errors='coerce')
earliest_date = df['Datum der Aufnahme'].min()
df['Days Since First Aufnahme'] = (df['Datum der Aufnahme'] - earliest_date).dt.days

# remove the Bezirksname, Kanton, PdA/Sol._2019
df.drop(columns=['Bezirksname', 'Kanton', 'PdA/Sol._2019', 'Datum der Aufnahme'], inplace=True)

# replace all values that are "X" with 0
df = df.map(lambda x: 0 if x == "X" else x).infer_objects(copy=False)

df

,Score,Hist.-Nummer,Bezirks-nummer,Gemeindecode,Einwohner_2019,Veränderung in %_2010-2019,Bevölkerungs-dichte pro km²_2019,Ausländer in %_2019,0-19 Jahre_2019,20-64 Jahre_2019,...,SP_2019,SVP_2019,EVP/CSP_2019,GLP_2019,BDP_2019,GPS_2019,Kleine Rechtsparteien_2019,Straftaten,percentage_first_sector,Days Since First Aufnahme
Gemeindename,,,,,,,,,,,,,,,,,,,,,
Bolken,42,13723.0,1106.0,2514,595.0,9.778598,280.660377,5.042017,23.025210,62.521008,...,20.050761,23.519459,0.338409,6.852792,4.822335,11.336717,0.084602,20,inf,51975
Hüniken,48,13733.0,1106.0,2524,153.0,70.000000,150.000000,3.921569,24.183007,56.209150,...,20.618557,23.711340,0.515464,6.958763,1.546392,16.752577,0.257732,5,inf,51975
Beinwil (SO),85,11876.0,1110.0,2612,275.0,-8.940397,12.135922,7.272727,22.545455,55.272727,...,4.668675,25.451807,0.150602,1.957831,0.150602,7.078313,1.204819,9,68.217054,0
Kammersrohr,82,12650.0,1107.0,2549,32.0,-17.948718,33.684211,3.125000,12.500000,62.500000,...,10.185185,11.111111,0.000000,10.185185,0.925926,7.407407,11.111111,1,66.666667,0
Kienberg,66,12691.0,1105.0,2492,504.0,0.000000,59.085580,8.730159,19.246032,60.515873,...,10.381978,37.708129,2.056807,2.938296,2.154750,6.268364,0.979432,17,51.694915,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuchwil,3,13743.0,1106.0,2534,9041.0,3.693084,1952.699784,43.689857,18.493529,63.200973,...,26.114263,20.674635,1.570097,8.032820,1.914506,12.753241,0.476094,789,0.124334,51975
Solothurn,14,10384.0,1109.0,2601,16933.0,5.396489,2696.337580,21.319317,15.644009,64.058348,...,24.851403,12.277776,0.944031,9.609747,1.750894,20.599231,0.317366,3861,0.064839,0
Olten,10,12486.0,1108.0,2581,18399.0,8.312239,1601.305483,29.278765,16.261753,65.106799,...,26.161744,16.126321,1.756124,8.693564,1.738112,20.097863,0.378242,2942,0.036337,0


These are the features that were used to create the original ranking in Assignment 1. Make sure these features are not in your dataset from now on. 
**TODO: Please write down your original features here!**

## Original Features
- Percentage of farmers -> percentage_first_sector
- Percentage of Swiss people -> 100 - (Ausländer in %_2019)
- Number of crimes -> Straftaten
- Population density -> Bevölkerungs-dichte pro km²_2019

## Related Features
As stated in the assignment, features that are related to the original features should also be removed. These are:
- Total employees -> Beschäftigte total_2018
- Employees in the first sector -> im 1. Sektor_2018
- Employees in the second sector -> im 2. Sektor_2018
- Employees in the third sector -> im 3. Sektor_2018
- Total number of workplaces -> Arbeitsstätten total_2018
- Workplaces in the first sector -> im 1. Sektor_2018.1
- Workplaces in the second sector -> im 2. Sektor_2018.1
- Workplaces in the third sector -> im 3. Sektor_2018.1

In [97]:
# TODO: Write down your original features here!

# removal of original features
df.drop(columns=['percentage_first_sector', 'Ausländer in %_2019', 'Straftaten', 'Bevölkerungs-dichte pro km²_2019'], inplace=True)

# remove of columns that were related to the original Features
df.drop(columns=['Beschäftigte total_2018', 'im 1. Sektor_2018', 'im 2. Sektor_2018', 'im 3. Sektor_2018', 'Arbeitsstätten total_2018', 'im 1. Sektor_2018.1', 'im 2. Sektor_2018.1', 'im 3. Sektor_2018.1'], inplace=True)

df

,Score,Hist.-Nummer,Bezirks-nummer,Gemeindecode,Einwohner_2019,Veränderung in %_2010-2019,0-19 Jahre_2019,20-64 Jahre_2019,65 Jahre und mehr_2019,Rohe Heiratssziffer_2019,...,FDP 2)_2019,CVP_2019,SP_2019,SVP_2019,EVP/CSP_2019,GLP_2019,BDP_2019,GPS_2019,Kleine Rechtsparteien_2019,Days Since First Aufnahme
Gemeindename,,,,,,,,,,,,,,,,,,,,,
Bolken,42,13723.0,1106.0,2514,595.0,9.778598,23.025210,62.521008,14.453782,1.672241,...,16.920474,13.620981,20.050761,23.519459,0.338409,6.852792,4.822335,11.336717,0.084602,51975
Hüniken,48,13733.0,1106.0,2524,153.0,70.000000,24.183007,56.209150,19.607843,0.000000,...,14.432990,14.948454,20.618557,23.711340,0.515464,6.958763,1.546392,16.752577,0.257732,51975
Beinwil (SO),85,11876.0,1110.0,2612,275.0,-8.940397,22.545455,55.272727,22.181818,0.000000,...,21.536145,37.198795,4.668675,25.451807,0.150602,1.957831,0.150602,7.078313,1.204819,0
Kammersrohr,82,12650.0,1107.0,2549,32.0,-17.948718,12.500000,62.500000,25.000000,0.000000,...,23.148148,24.074074,10.185185,11.111111,0.000000,10.185185,0.925926,7.407407,11.111111,0
Kienberg,66,12691.0,1105.0,2492,504.0,0.000000,19.246032,60.515873,20.238095,1.966568,...,17.629775,17.825661,10.381978,37.708129,2.056807,2.938296,2.154750,6.268364,0.979432,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuchwil,3,13743.0,1106.0,2534,9041.0,3.693084,18.493529,63.200973,18.305497,3.332778,...,15.498379,10.615883,26.114263,20.674635,1.570097,8.032820,1.914506,12.753241,0.476094,51975
Solothurn,14,10384.0,1109.0,2601,16933.0,5.396489,15.644009,64.058348,20.297644,5.695639,...,19.743955,8.767919,24.851403,12.277776,0.944031,9.609747,1.750894,20.599231,0.317366,0
Olten,10,12486.0,1108.0,2581,18399.0,8.312239,16.261753,65.106799,18.631447,7.072521,...,15.751081,8.324328,26.161744,16.126321,1.756124,8.693564,1.738112,20.097863,0.378242,0


# Get a first baseline for your model with random values

In [98]:
# This code creates a baseline for your model, using random features
createBaselineRandom(df, 'Score')

linear   : -461.9609593322414
boosting : -536.5381768044549


**WARNING**: This uses the same dataframe for both the linear model and the boosting model. In your code you will want to use different ones for the different models. Your goal is to get both of these scores to as close as 0 as possible and most likely you will need to do different steps for linear regression and for gradient boosting and therefore have different features. 

# Get a second baseline by using all features you have

In [99]:
# this assumes we have a dataframe with a column named Score and all other columns are features.
# You might have to change this code if your dataframe looks different.

linear, boost = runScoringSimple(df, 'Score')
print(f'linear   : {linear}')
print(f'boosting : {boost}')

linear   : -191.1860500666951
boosting : -147.83943188917434


# Extend your features and add something of a geographic nature

Take the features you have loaded above and potentially extend it with the data from the gemeindeporträts 2021. Now you can extend this with some kind of geographic information. This can be pretty much anything that uses coordinates, height, relative position to other features etc. Try to find something reasonable that might help you, but don't worry if you find out during your feature engineering that it is not helpful, you are not required to use the feature for your final model.

# Get the coordinates of the Gemeinden
Use the API from geo.admin.ch to get the coordinates of the Gemeinden. These coordinates will them be used to create a new geographic feature.

In [102]:
API_URL = 'https://api3.geo.admin.ch/rest/services/api/SearchServer'

latitudes = []
longitudes = []

# Get the coordinates of each of the Gemeinden.
for name in df.index:
    params = {
        'type': 'locations',
        'origins': 'gg25',
        'searchText': str(name),
        'sr': '4326', # WGS 84
        'limit': 1 # Get the top Result
    }
    # Make the API call
    result = requests.get(API_URL, params=params).json()

    # Add the coordinates to the List
    if result.get('results'):
        attrs = result['results'][0]['attrs']
        longitudes.append(attrs['x'])
        latitudes.append(attrs['y'])
    # Error handling - should never be called
    else:
        longitudes.append(None)
        latitudes.append(None)

# Add the coordinates to the dataframe
df['Longitude'] = longitudes
df['Latitude'] = latitudes

df


,Score,Hist.-Nummer,Bezirks-nummer,Gemeindecode,Einwohner_2019,Veränderung in %_2010-2019,0-19 Jahre_2019,20-64 Jahre_2019,65 Jahre und mehr_2019,Rohe Heiratssziffer_2019,...,SP_2019,SVP_2019,EVP/CSP_2019,GLP_2019,BDP_2019,GPS_2019,Kleine Rechtsparteien_2019,Days Since First Aufnahme,Longitude,Latitude
Gemeindename,,,,,,,,,,,,,,,,,,,,,
Bolken,42,13723.0,1106.0,2514,595.0,9.778598,23.025210,62.521008,14.453782,1.672241,...,20.050761,23.519459,0.338409,6.852792,4.822335,11.336717,0.084602,51975,7.663737,47.193565
Hüniken,48,13733.0,1106.0,2524,153.0,70.000000,24.183007,56.209150,19.607843,0.000000,...,20.618557,23.711340,0.515464,6.958763,1.546392,16.752577,0.257732,51975,7.636059,47.183060
Beinwil (SO),85,11876.0,1110.0,2612,275.0,-8.940397,22.545455,55.272727,22.181818,0.000000,...,4.668675,25.451807,0.150602,1.957831,0.150602,7.078313,1.204819,0,7.589023,47.354610
Kammersrohr,82,12650.0,1107.0,2549,32.0,-17.948718,12.500000,62.500000,25.000000,0.000000,...,10.185185,11.111111,0.000000,10.185185,0.925926,7.407407,11.111111,0,7.591982,47.256771
Kienberg,66,12691.0,1105.0,2492,504.0,0.000000,19.246032,60.515873,20.238095,1.966568,...,10.381978,37.708129,2.056807,2.938296,2.154750,6.268364,0.979432,0,7.965604,47.442192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuchwil,3,13743.0,1106.0,2534,9041.0,3.693084,18.493529,63.200973,18.305497,3.332778,...,26.114263,20.674635,1.570097,8.032820,1.914506,12.753241,0.476094,51975,7.562047,47.206112
Solothurn,14,10384.0,1109.0,2601,16933.0,5.396489,15.644009,64.058348,20.297644,5.695639,...,24.851403,12.277776,0.944031,9.609747,1.750894,20.599231,0.317366,0,7.529796,47.208057
Olten,10,12486.0,1108.0,2581,18399.0,8.312239,16.261753,65.106799,18.631447,7.072521,...,26.161744,16.126321,1.756124,8.693564,1.738112,20.097863,0.378242,0,7.893518,47.339989


# Distance to nearest Asylzentrum


In [118]:
import requests
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Function to calculate the distance between two points using the Haversine formula
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    # Radius of Earth in kilometers
    radius = 6371
    distance = radius * c

    return distance

# Get the asylum centers data
def get_asylum_centers():
    asylum_api_url = 'https://api3.geo.admin.ch/rest/services/api/SearchServer'
    params = {
        'features': 'ch.sem.sachplan-asyl_kraft',
        'type': 'locations',
        'searchText': 'Bundesasylzentrum'
    }

    response = requests.get(asylum_api_url, params=params).json()

    # Extract the coordinates and names
    asylum_centers = []
    if 'results' in response:
        for result in response['results']:
            attrs = result.get('attrs', {})
            if 'x' in attrs and 'y' in attrs and 'label' in attrs:
                asylum_centers.append({
                    'name': attrs['label'],
                    'longitude': attrs['x'],
                    'latitude': attrs['y']
                })

    return pd.DataFrame(asylum_centers)

# Get asylum centers
asylum_centers_df = get_asylum_centers()
print(f"Found {len(asylum_centers_df)} asylum centers")

# Calculate the distance to the nearest asylum center for each municipality
def add_nearest_asylum_center_distance(df, asylum_centers):
    # Create a new column for the minimum distance
    df['Distance_to_nearest_asylum_center'] = None
    df['Nearest_asylum_center'] = None

    # For each municipality
    for idx, row in df.iterrows():
        min_distance = float('inf')
        nearest_center = None

        # Calculate distance to each asylum center
        for _, center in asylum_centers.iterrows():
            distance = haversine_distance(
                row['Latitude'], row['Longitude'],
                center['latitude'], center['longitude']
            )

            if distance < min_distance:
                min_distance = distance
                nearest_center = center['name']

        # Update the dataframe
        df.at[idx, 'Distance_to_nearest_asylum_center'] = min_distance
        df.at[idx, 'Nearest_asylum_center'] = nearest_center

    return df

# Add the feature to our dataframe
df = add_nearest_asylum_center_distance(df, asylum_centers_df)

# Display the result
print("\nDataframe with new asylum center distance feature:")
display(df[['Longitude', 'Latitude', 'Distance_to_nearest_asylum_center', 'Nearest_asylum_center']].head())


Found 50 asylum centers

Dataframe with new asylum center distance feature:


,Longitude,Latitude,Distance_to_nearest_asylum_center,Nearest_asylum_center
Gemeindename,,,,
Bolken,7.663737,47.193565,920.69538,<b>Pont-la-Ville (FR)</b>
Hüniken,7.636059,47.183060,922.566923,<b>Pont-la-Ville (FR)</b>
Beinwil (SO),7.589023,47.354610,906.307981,<b>Pont-la-Ville (FR)</b>
Kammersrohr,7.591982,47.256771,916.257481,<b>Pont-la-Ville (FR)</b>
Kienberg,7.965604,47.442192,886.55195,<b>Pont-la-Ville (FR)</b>


In [119]:
asylum_centers_df

,name,longitude,latitude
0,<b>Bonaduz (GR)</b>,185282.250000,746111.5000
1,<b>Les Ponts-de-Martel (NE)</b>,205406.406250,546172.4375
2,<b>Pont-en-Ogoz (FR)</b>,170836.343750,572314.0000
3,<b>Pont-la-Ville (FR)</b>,172453.062500,574974.7500
4,<b>La Punt Chamues-ch (GR)</b>,157257.156250,793373.6875
5,<b>Ponte Capriasca (TI)</b>,109369.804688,725771.8750
6,<i>Ort</i> <b>Bonaduz</b> (GR) - Bonaduz,186283.515625,749308.6250
7,<i>Ort</i> <b>Les Ponts-de-Martel</b> (NE) - L...,205475.187500,546117.9375
8,<i>Ort</i> <b>Les Favarges</b> (NE) - Les Pont...,204225.187500,543892.6875
9,<i>Ort</i> <b>Martel-Dernier</b> (NE) - Les Po...,203956.734375,544520.2500


# Do your own feature engineering here

Your goal is to find the right features to get both scores as close to 0 as possible. Add more columns to your dataframe through feature engineering and choose which ones of those you will use for the machine learning models. Use the methods discussed in class to improve your results even further.

In [86]:
#TODO: Insert your code here 